In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import datasets

In [ ]:
sub_df = pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/sample_submission.csv")
val_df = pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv")
score_df = pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv")

In [ ]:
sub_df.head(2)

In [ ]:
score_df.head(2)

In [ ]:
val_df.head(), val_df.shape

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/roberta-toxicity-classifier/")
model = AutoModelForSequenceClassification.from_pretrained("/kaggle/input/roberta-toxicity-classifier/")

In [ ]:
data = datasets.load_dataset('csv', data_files=['/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv'])

In [ ]:
tok = tokenizer('I love food', return_tensors='pt')
print(tok)

In [ ]:
out = model(**tok)

In [ ]:
torch.softmax(out[0], dim=1)[0][1].item()

In [ ]:
out.logits.sum(dim=1), out.logits[:,1]

In [ ]:
def tokenize_data(example):
    inputs = tokenizer(example['text'], padding=True, truncation=True)
#     inputs2 = tokenizer(example['more_toxic'], padding=True, truncation=True)
    return inputs

In [ ]:
data_enc = data['train'].map(tokenize_data, batched=True)

data_enc.set_format(type='torch', columns = ['input_ids',
                                         'attention_mask'])

In [ ]:
data_enc.column_names

In [ ]:
data_enc['input_ids'][0].size(), data_enc['attention_mask'][0].shape

In [ ]:
def prediction(model, dataset_enc):
    model.to('cuda')
    model.eval()
    data_loader = torch.utils.data.DataLoader(dataset_enc, batch_size=32)
    outputs = []
    import pdb
    for ixd, loader in enumerate(data_loader):
        with torch.no_grad():
            output = model(input_ids=loader['input_ids'].to('cuda'), 
                           attention_mask=loader['attention_mask'].to("cuda"))
#             pdb.set_trace()
            output = torch.softmax(output.logits, dim=-1)[:,1]
        
            output = output.detach().cpu().numpy().tolist()
            outputs.extend(output)
    torch.cuda.empty_cache()
    return np.array(outputs)
#     break

In [ ]:
pred = prediction(model, data_enc)

In [ ]:
pred.shape

In [ ]:
score_df.shape

In [ ]:
score_df['score'] = pred

In [ ]:
score_df['score'] = score_df['score'].rank(method='first')

In [ ]:
score_df[['comment_id', 'score']].to_csv('submission.csv', index=False)